In [1]:
using Revise

In [2]:
using RigidBodyDynamics
using RigidBodyDynamics: source, target
using RigidBodyDynamics.PDControl
using StaticArrays

In [3]:
urdf = Pkg.dir("RigidBodyDynamics", "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(Float64, urdf)

state = MechanismState(robot)
set_velocity!(state, randn(2))
set_configuration!(state, randn(2))

true

In [4]:
effector_body = findbody(robot, "lower_link")
effector_pt = Point3D(default_frame(effector_body), 0., 0, -2)

Point3D in "after_elbow": [0.0, 0.0, -2.0]

In [5]:
p = path(robot, root_body(robot), effector_body)

Path from world to lower_link:
↓ base_link_to_world
↓ shoulder
↓ elbow


In [6]:
using MeshCatMechanisms
using Blink

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [7]:
vis = MechanismVisualizer(robot, URDFVisuals(urdf))

setelement!(vis, effector_pt, 0.07)
set_configuration!(vis, configuration(state))

open(vis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/.julia/v0.6/Blink/deps/atom/electron /home/rdeits/.julia/v0.6/Blink/src/AtomShell/main.js port 4091`, ProcessRunning), TCPSocket(RawFD(51) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(56) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 1, Nullable{Any}(true))))

In [8]:
using MeshCat
using GeometryTypes: Point

In [9]:
θ = repeat(linspace(0, 2π, 100), inner=(2,))[2:end]
geometry = PointCloud(Point.(0.5 .* sin.(θ), 0.25, 2 .+ 0.5 .* cos.(θ)))
delete!(vis[:circle])
setobject!(vis[:circle], LineSegments(geometry, LineBasicMaterial()))

MeshCat Visualizer with path /meshcat/circle

In [10]:
Jp = point_jacobian(state, p, Point3D(root_frame(robot), 0, 0, 0))
α = 100
dt = 1e-3
for i in 1:5000
    desired = [0.5 * sin(i / 300), 0, 2 + 0.5 * cos(i / 300)]
    effector_pt_in_world = transform(state, effector_pt, root_frame(robot))
    point_jacobian!(Jp, state, p, effector_pt_in_world)
    vdes = α * Array(Jp)' * (desired - effector_pt_in_world.v)
    set_velocity!(state, vdes)
    set_configuration!(state, configuration(state) + dt * velocity(state))
    set_configuration!(vis, configuration(state))
    sleep(dt)
end

In [11]:
rand!(state)

true

In [12]:
function controller!(τ, t, state)
    desired = [0.5 * sin(t), 0, 2 + 0.5 * cos(t)]
    effector_pt_in_world = transform(state, effector_pt, root_frame(robot))
    point_jacobian!(Jp, state, p, effector_pt_in_world)
    v̇ = similar(velocity(state))
    v̇ .= 200 * Array(Jp)' * (desired - effector_pt_in_world.v) .- 20 .* velocity(state)
    τ .= inverse_dynamics(state, v̇)
end

controller! (generic function with 1 method)

In [13]:
ts, qs, vs = simulate(state, 10, controller!);

In [14]:
setanimation!(vis, ts, qs)

true

In [101]:
gains = RigidBodyDynamics.PDControl.PDGains(10, 1)
H = transform_to_root(state, effector_body)
T = transform(twist_wrt_world(state, effector_body), inv(H))
a = PDControl.pd(gains, translation(H) - [1, 0, 1], linear(T))

Ṫ_desired = SpatialAcceleration(T.body, T.base, T.frame, zero(a), a)

SpatialAcceleration of "after_elbow" w.r.t "world" in "after_elbow":
angular: [0.0, 0.0, 0.0], linear: [8.81691, -2.5, -0.0578644]

In [102]:
function point_acceleration(state, v̇, body)
    p = path(state.mechanism, root_body(state.mechanism), body)
    J = geometric_jacobian(state, p)
    J̇v = -bias_acceleration(state, source(p)) + bias_acceleration(state, target(p))
    SpatialAcceleration(J, v̇) + J̇v
end

point_acceleration (generic function with 1 method)

In [103]:
transform(state,
          -point_acceleration(state, [0, 1], effector_body),
          Ṫ_desired.frame) + Ṫ_desired

SpatialAcceleration of "after_elbow" w.r.t "after_elbow" in "after_elbow":
angular: [0.0, -1.0, 0.0], linear: [8.83602, -2.5, -0.0555152]

In [104]:
using JuMP
using OSQP

In [134]:
set_configuration!(state, randn(2))
set_velocity!(state, randn(2))

true

In [133]:
function controller!(τ::AbstractVector, state::MechanismState, body, target=[-1, 0, -1])
    solver = OSQPMathProgBaseInterface.OSQPSolver(eps_abs = 1e-8, eps_rel = 1e-16, max_iter = 10000)
    MathProgBase.setparameters!(solver, Silent = true)
    model = JuMP.Model(solver=solver)
    @variable model v̇[1:2]
    obj = dot(v̇, v̇)
    
    gains = RigidBodyDynamics.PDControl.PDGains(1, 0.5)
    H = transform_to_root(state, body)
    T = transform(twist_wrt_world(state, body), inv(H))
    a = PDControl.pd(gains, translation(H) - target, linear(T))

    Ṫ_desired = SpatialAcceleration(T.body, T.base, T.frame, zero(a), a)

    error = transform(state,
              -point_acceleration(state, v̇, body),
              Ṫ_desired.frame) + Ṫ_desired
    obj += 10 * dot(error.linear, error.linear)
    @objective model Min obj
    solve(model)
    v̇_out = similar(velocity(state))
    v̇_out .= getvalue(v̇)
    τ .= inverse_dynamics(state, v̇_out)
end
    

controller! (generic function with 4 methods)

In [136]:
ts, qs, vs = simulate(state, 
    5, 
    (τ, t, state) -> controller!(τ, state, effector_body),
    Δt=1e-3);

setanimation!(vis, ts, qs)

true